# Johnson-Lindenstrauss (JL) Dimensionality Reduction + ANN (Optimized)

This notebook evaluates approximate nearest neighbor search using sklearn's optimized Johnson-Lindenstrauss random projection, followed by brute-force search in the lower-dimensional space.

## Method
1. Apply JL projection using sklearn.random_projection.SparseRandomProjection
2. Build KNN index in k-dimensional space using sklearn
3. Project query vectors and search in k-dimensional space

## Key Advantages
- ✅ Uses sklearn's highly optimized C/Cython implementation
- ✅ Sparse random matrices (Achlioptas 2001) for efficiency
- ✅ Automatic parameter selection based on JL lemma
- ✅ Fast matrix operations with BLAS/LAPACK

## Metrics
- **Recall@10**: Proportion of true nearest neighbors retrieved
- **Query time**: Time per query (including projection) in milliseconds
- **Build time**: Time to construct projection + index
- **Memory**: Total memory footprint in MB

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.random_projection import SparseRandomProjection
from sklearn.neighbors import NearestNeighbors
import time
import sys
import json
import os

# Import our custom modules
sys.path.append('..')  # If running from notebooks/ subdirectory
from evaluator import ANNEvaluator
from datasets import DatasetLoader, print_dataset_info

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 10)

print("✓ Imports successful!")
print(f"Using sklearn's optimized SparseRandomProjection")

✓ Imports successful!
Using sklearn's optimized SparseRandomProjection


## 1. Configuration

In [18]:
# Configuration
K = 10  # Number of nearest neighbors
DATA_DIR = "data"  # Adjust if needed

# Which datasets to evaluate
DATASETS = [
    'sift',
    'gist',
    'deep1b'
]

# Target dimensions - optimized based on dataset characteristics
TARGET_DIMS = {
    'sift': [32, 64, 80, 90, 100, 120, 124],           # 128-dim → 2-4x compression
    'gist': [100, 200, 300, 500, 700],      # 960-dim → 1.4-10x compression
    'deep1b': [32, 48, 64, 80]              # 96-dim → 1.2-3x compression
}

# Subset sizes
N_TRAIN = None  # Use full training set
N_TEST = 1000   # Use 1000 test queries

# JL parameters for sklearn
JL_EPS = 0.1        # Distance distortion parameter
JL_DENSITY = 'auto' # Sparsity: 'auto' uses 1/sqrt(n_components)
RANDOM_STATE = 42

# Results directory
RESULTS_DIR = "../results"
os.makedirs(RESULTS_DIR, exist_ok=True)

print(f"Configuration:")
print(f"  k (neighbors) = {K}")
print(f"  JL epsilon = {JL_EPS}")
print(f"  Datasets: {DATASETS}")
print(f"\nTarget dimensions:")
for ds, dims in TARGET_DIMS.items():
    print(f"  {ds.upper()}: {dims}")

Configuration:
  k (neighbors) = 10
  JL epsilon = 0.1
  Datasets: ['sift', 'gist', 'deep1b']

Target dimensions:
  SIFT: [32, 64, 80, 90, 100, 120, 124]
  GIST: [100, 200, 300, 500, 700]
  DEEP1B: [32, 48, 64, 80]


## 2. Sklearn-Based JL+KNN Implementation

Using sklearn's production-ready implementations:
- **SparseRandomProjection**: Optimized C/Cython implementation
- **NearestNeighbors**: Fast brute-force search with parallel execution

In [19]:
class SklearnJLKNN:
    """
    JL+KNN using sklearn's optimized implementations.
    Much faster than custom implementation.
    """
    def __init__(self, n_components, eps=0.1, density='auto', n_jobs=-1, random_state=42):
        """
        Args:
            n_components: Target dimension k
            eps: JL epsilon parameter for distance preservation
            density: Sparsity of projection matrix ('auto' recommended)
            n_jobs: Number of CPU cores (-1 = all)
            random_state: Random seed for reproducibility
        """
        self.n_components = n_components
        self.eps = eps
        self.n_jobs = n_jobs
        
        # Initialize projector
        self.projector = SparseRandomProjection(
            n_components=n_components,
            eps=eps,
            density=density,
            random_state=random_state
        )
        
        # Initialize KNN
        self.knn = NearestNeighbors(
            algorithm='brute',
            metric='euclidean',
            n_jobs=n_jobs
        )
        
        self.X_train_projected = None
        
    def fit(self, X_train):
        """Build projection and KNN index"""
        print(f"  Projecting {X_train.shape[0]:,} points: {X_train.shape[1]}D → {self.n_components}D...")
        
        # Project training data (sklearn handles this efficiently)
        start = time.time()
        self.X_train_projected = self.projector.fit_transform(X_train)
        proj_time = time.time() - start
        
        print(f"    Projection time: {proj_time:.2f}s")
        print(f"    Projected shape: {self.X_train_projected.shape}")
        print(f"    Memory: {self.X_train_projected.nbytes / (1024**2):.2f} MB")
        
        # Build KNN index on projected data
        start = time.time()
        self.knn.fit(self.X_train_projected)
        knn_time = time.time() - start
        
        print(f"    KNN index time: {knn_time:.2f}s")
        
        return self
    
    def query(self, X_test, k):
        """Query KNN in projected space"""
        # Project queries (fast with sklearn)
        X_test_projected = self.projector.transform(X_test)
        
        # Query KNN
        distances, indices = self.knn.kneighbors(X_test_projected, n_neighbors=k)
        
        return indices, distances
    
    def get_memory_usage(self):
        """Estimate memory usage in MB"""
        # Projected data
        data_mem = self.X_train_projected.nbytes / (1024**2)
        
        # Projection matrix (sparse)
        if hasattr(self.projector, 'components_'):
            proj_mem = self.projector.components_.data.nbytes / (1024**2)
        else:
            proj_mem = 0
        
        return data_mem + proj_mem


def build_sklearn_jl_index(X_train, n_components, eps=0.1, n_jobs=-1):
    """Wrapper for evaluator"""
    model = SklearnJLKNN(
        n_components=n_components,
        eps=eps,
        n_jobs=n_jobs,
        random_state=RANDOM_STATE
    )
    model.fit(X_train)
    return model


def query_sklearn_jl_index(index, X_test, k):
    """Wrapper for evaluator"""
    return index.query(X_test, k)


print("✓ Sklearn JL+KNN implementation ready!")

✓ Sklearn JL+KNN implementation ready!


## 3. Run JL Experiments

In [20]:
# Store all results
all_results = {}

# Initialize dataset loader
loader = DatasetLoader(data_dir=DATA_DIR)

for dataset_name in DATASETS:
    print(f"\n{'='*70}")
    print(f"DATASET: {dataset_name.upper()}")
    print(f"{'='*70}")
    
    # Print dataset info
    print_dataset_info(dataset_name)
    
    # Load dataset
    try:
        X_train, X_test = loader.load_dataset(
            dataset_name,
            n_train=N_TRAIN,
            n_test=N_TEST
        )
    except FileNotFoundError as e:
        print(f"ERROR: Could not load {dataset_name} dataset.")
        print(f"  {e}")
        print(f"  Skipping this dataset...\n")
        continue
    
    d_original = X_train.shape[1]
    
    # Initialize evaluator
    evaluator = ANNEvaluator(X_train, X_test, k=K)
    
    # Compute ground truth
    evaluator.compute_ground_truth()
    
    # Store results for this dataset
    dataset_results = []
    
    # Sweep over target dimensions
    for target_dim in TARGET_DIMS[dataset_name]:
        if target_dim >= d_original:
            print(f"\nSkipping k={target_dim} (≥ original dimension {d_original})")
            continue
            
        print(f"\n{'='*60}")
        print(f"Evaluating: JL-KNN (d={d_original} → k={target_dim})")
        print(f"  Compression ratio: {d_original/target_dim:.2f}x")
        print(f"{'='*60}")
        
        # Create index builder
        def build_fn(X):
            return build_sklearn_jl_index(X, target_dim, eps=JL_EPS)
        
        # Evaluate
        try:
            results = evaluator.evaluate(
                index_builder=build_fn,
                query_func=query_sklearn_jl_index,
                method_name=f"JL-KNN-{dataset_name.upper()}-k{target_dim}"
            )
            
            # Add metadata
            results['dataset'] = dataset_name
            results['d_original'] = d_original
            results['k_projected'] = target_dim
            results['compression_ratio'] = d_original / target_dim
            results['jl_epsilon'] = JL_EPS
            
            dataset_results.append(results)
            
            # Save individual result
            result_path = os.path.join(RESULTS_DIR, f"jl_sklearn_{dataset_name}_k{target_dim}.json")
            with open(result_path, 'w') as f:
                json.dump(results, f, indent=2)
            
            print(f"\n✓ Results saved to {result_path}")
            
        except Exception as e:
            print(f"ERROR during evaluation: {e}")
            import traceback
            traceback.print_exc()
            continue
    
    # Store results for this dataset
    all_results[dataset_name] = dataset_results

print(f"\n{'='*70}")
print("ALL JL EXPERIMENTS COMPLETE")
print(f"{'='*70}\n")


DATASET: SIFT

SIFT-1M Dataset Info:
  Description: SIFT image descriptors
  Dimension: 128
  Training size: 1,000,000
  Test size: 10,000

Loading SIFT-1M dataset...
  Training: (1000000, 128) (dtype: float32)
  Test: (1000, 128) (dtype: float32)
Evaluator initialized:
  Training points: 1,000,000
  Test queries: 1,000
  Dimensions: 128
  k (neighbors): 10
Computing ground truth k-NN (k=10) using brute force...
Ground truth computed in 2.18s
Ground truth shape: (1000, 10)

Evaluating: JL-KNN (d=128 → k=32)
  Compression ratio: 4.00x

Evaluating: JL-KNN-SIFT-k32
Building index...
  Projecting 1,000,000 points: 128D → 32D...
    Projection time: 0.59s
    Projected shape: (1000000, 32)
    Memory: 122.07 MB
    KNN index time: 0.04s
  Build time: 0.63s
  Memory used: 244.14 MB
Querying 1,000 test points...
  Avg query time: 0.982 ms
  Recall@10: 0.1800 (18.00%)
  Recall stats: mean=0.1800, std=0.1673, min=0.0000, max=0.9000


✓ Results saved to ../results/jl_sklearn_sift_k32.json

Eval

KeyboardInterrupt: 

## 4. Visualize Results

In [ ]:
# Create per-dataset plots
for dataset_name, results_list in all_results.items():
    if not results_list:
        continue
    
    print(f"\nPlotting results for {dataset_name.upper()}...")
    
    # Extract data
    target_dims = [r['k_projected'] for r in results_list]
    recalls = [r['recall@k'] for r in results_list]
    query_times = [r['avg_query_time_ms'] for r in results_list]
    memories = [r['memory_mb'] for r in results_list]
    compression_ratios = [r['compression_ratio'] for r in results_list]
    d_orig = results_list[0]['d_original']
    
    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'JL+KNN Results (sklearn): {dataset_name.upper()} (d={d_orig})', 
                 fontsize=16, fontweight='bold')
    
    # 1. Recall vs Target Dimension
    axes[0, 0].plot(target_dims, recalls, marker='o', linewidth=2.5, markersize=10, color='steelblue')
    axes[0, 0].axhline(y=1.0, color='green', linestyle='--', alpha=0.5, label='Perfect Recall')
    axes[0, 0].axhline(y=0.90, color='orange', linestyle='--', alpha=0.5, label='90% Recall Target')
    axes[0, 0].set_xlabel('Target Dimension k', fontsize=11, fontweight='bold')
    axes[0, 0].set_ylabel(f'Recall@{K}', fontsize=11, fontweight='bold')
    axes[0, 0].set_title('Recall vs. Dimension', fontsize=12, fontweight='bold')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].legend()
    axes[0, 0].set_ylim([max(0.5, min(recalls) - 0.05), 1.02])
    
    # Add annotations
    for i, (k, r) in enumerate(zip(target_dims, recalls)):
        if r >= 0.90:
            axes[0, 0].annotate(f'{r:.2f}', (k, r), textcoords="offset points", 
                               xytext=(0,5), ha='center', fontsize=9, color='green', fontweight='bold')
    
    # 2. Query Time vs Target Dimension  
    axes[0, 1].plot(target_dims, query_times, marker='s', linewidth=2.5, markersize=10, color='coral')
    axes[0, 1].set_xlabel('Target Dimension k', fontsize=11, fontweight='bold')
    axes[0, 1].set_ylabel('Query Time (ms)', fontsize=11, fontweight='bold')
    axes[0, 1].set_title('Query Time vs. Dimension', fontsize=12, fontweight='bold')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Compression Ratio vs Recall
    axes[1, 0].plot(compression_ratios, recalls, marker='^', linewidth=2.5, markersize=10, 
                   color='mediumseagreen')
    axes[1, 0].axhline(y=0.90, color='orange', linestyle='--', alpha=0.5, label='90% Target')
    axes[1, 0].set_xlabel('Compression Ratio (d/k)', fontsize=11, fontweight='bold')
    axes[1, 0].set_ylabel(f'Recall@{K}', fontsize=11, fontweight='bold')
    axes[1, 0].set_title('Recall vs. Compression Ratio', fontsize=12, fontweight='bold')
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].legend()
    
    # 4. Recall-Query Time Pareto Frontier
    scatter = axes[1, 1].scatter(query_times, recalls, s=150, c=target_dims, 
                                cmap='viridis', alpha=0.7, edgecolors='black', linewidth=2)
    # Connect points
    axes[1, 1].plot(query_times, recalls, 'k--', alpha=0.3, linewidth=1)
    
    # Annotate points
    for i, k in enumerate(target_dims):
        axes[1, 1].annotate(f'k={k}', (query_times[i], recalls[i]), 
                           textcoords="offset points", xytext=(5,5), ha='left', fontsize=9)
    
    axes[1, 1].set_xlabel('Query Time (ms)', fontsize=11, fontweight='bold')
    axes[1, 1].set_ylabel(f'Recall@{K}', fontsize=11, fontweight='bold')
    axes[1, 1].set_title('Pareto Frontier: Recall vs Speed', fontsize=12, fontweight='bold')
    axes[1, 1].grid(True, alpha=0.3)
    cbar = plt.colorbar(scatter, ax=axes[1, 1])
    cbar.set_label('Target Dimension k', fontsize=10)
    
    plt.tight_layout()
    plot_path = os.path.join(RESULTS_DIR, f'jl_sklearn_{dataset_name}_tradeoffs.png')
    plt.savefig(plot_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"✓ Plot saved to {plot_path}")

## 5. Summary Statistics

In [ ]:
# Flatten all results
all_results_flat = []
for dataset_name, results_list in all_results.items():
    all_results_flat.extend(results_list)

if all_results_flat:
    print("\n" + "="*100)
    print("SUMMARY: JL+KNN (sklearn optimized)")
    print("="*100)
    
    # Header
    print(f"{'Dataset':<10} {'d→k':<15} {'Ratio':<8} {'Recall@10':<12} {'Query(ms)':<12} {'Memory(MB)':<12}")
    print("-" * 100)
    
    # Sort by dataset, then target dimension
    sorted_results = sorted(all_results_flat, key=lambda x: (x['dataset'], x['k_projected']))
    
    for r in sorted_results:
        dim_str = f"{r['d_original']}→{r['k_projected']}"
        ratio_str = f"{r['compression_ratio']:.2f}x"
        recall_str = f"{r['recall@k']:.4f}"
        query_str = f"{r['avg_query_time_ms']:.3f}"
        mem_str = f"{r['memory_mb']:.3f}"
        
        # Highlight good results (recall >= 0.90)
        marker = "✓" if r['recall@k'] >= 0.90 else " "
        
        print(f"{marker} {r['dataset'].upper():<9} {dim_str:<15} {ratio_str:<8} "
              f"{recall_str:<12} {query_str:<12} {mem_str:<12}")
    
    # Save summary
    summary = {
        'experiment': 'JL + KNN (sklearn)',
        'k': K,
        'n_test': N_TEST,
        'jl_epsilon': JL_EPS,
        'library': 'sklearn.random_projection.SparseRandomProjection',
        'configurations_tested': len(all_results_flat),
        'results_by_dataset': all_results
    }
    
    summary_path = os.path.join(RESULTS_DIR, 'jl_sklearn_summary.json')
    with open(summary_path, 'w') as f:
        json.dump(summary, f, indent=2)
    
    print(f"\n✓ Summary saved to {summary_path}")

## 6. Best Configurations

In [ ]:
if all_results:
    print("\n" + "="*80)
    print("BEST CONFIGURATIONS (Target: Recall ≥ 0.90)")
    print("="*80 + "\n")
    
    for dataset_name, results_list in all_results.items():
        if not results_list:
            continue
        
        print(f"\n{dataset_name.upper()}:")
        print("-" * 60)
        
        # Filter high-recall configs (>= 0.90)
        high_recall = [r for r in results_list if r['recall@k'] >= 0.90]
        
        if high_recall:
            # Best compression with high recall
            best_compression = max(high_recall, key=lambda x: x['compression_ratio'])
            print(f"\n  Best Compression (Recall ≥ 0.90):")
            print(f"    {best_compression['d_original']}D → {best_compression['k_projected']}D "
                  f"({best_compression['compression_ratio']:.2f}x compression)")
            print(f"    Recall@{K}: {best_compression['recall@k']:.4f}")
            print(f"    Query time: {best_compression['avg_query_time_ms']:.3f} ms")
            
            # Fastest with high recall
            fastest = min(high_recall, key=lambda x: x['avg_query_time_ms'])
            print(f"\n  Fastest Query (Recall ≥ 0.90):")
            print(f"    {fastest['d_original']}D → {fastest['k_projected']}D")
            print(f"    Recall@{K}: {fastest['recall@k']:.4f}")
            print(f"    Query time: {fastest['avg_query_time_ms']:.3f} ms")
        else:
            print(f"\n  ⚠ No configurations achieved Recall ≥ 0.90")
            best = max(results_list, key=lambda x: x['recall@k'])
            print(f"    Best recall: {best['recall@k']:.4f} at k={best['k_projected']}")
        
        print()

## 7. Key Observations

**sklearn Implementation Advantages:**
- ✅ Highly optimized C/Cython code
- ✅ Sparse random matrices for memory efficiency
- ✅ Parallel query execution
- ✅ Automatic parameter validation

**Expected Performance:**
- High recall (≥0.95) should be achievable with moderate compression (2-3x)
- Query time should scale with target dimension k
- Best results on high-dimensional data (GIST)

**Next Steps:**
1. Compare against baseline brute-force KNN
2. Compare against optimized LSH implementation
3. Analyze when JL provides the best tradeoffs